## Notebook Summary

This notebook performs an initial exploration and visualization of the 3W Dataset 2.0.0 for a selected class. The key steps and findings are as follows:

1.  **Package Installation:** Required libraries, including `pandas-profiling` and `ydata-profiling`, were installed to facilitate data analysis and profiling.
2.  **Data Loading:** The script cloned the 3W Dataset repository and identified all parquet files for each of the ten classes (0-9).
3.  **Data Stacking (for a chosen class):** For a specific chosen class (in this case, Class 2: SPURIOUS_CLOSURE_OF_DHSV), all the corresponding parquet files were loaded and concatenated into a single pandas DataFrame (`df_combined`). This combined DataFrame contains 737,785 rows and 29 columns.
4.  **Data Profiling:** A comprehensive data profiling report was generated for the combined DataFrame of the chosen class using `ydata-profiling`. This report, saved as an HTML file, provides insights into the data's structure, missing values, descriptive statistics, and potential issues.
5.  **Data Visualization (Single Instance):** To understand the time series behavior of the data, a single instance (parquet file) from the chosen class was loaded into a DataFrame (`df`).
6.  **Filtered Numeric Column Selection:** Numeric columns within the single-instance DataFrame were filtered to include only those with a significant proportion of non-missing values (controlled by a threshold).
7.  **Linked Subplots Visualization:** Linked Plotly subplots were generated for the selected valid numeric columns of the single instance. This allows for interactive exploration and synchronized zooming across different features over time.
8.  **Normalized Single Plot Visualization:** The selected valid numeric columns of the single instance were normalized using `MinMaxScaler`, and a single Plotly plot was created to visualize their trends on a comparable scale.

Overall, this notebook demonstrates the process of loading, combining, profiling, and visualizing a subset of the 3W Dataset, focusing on a specific class and a single instance to gain initial insights into the data's characteristics and time series patterns.

# Install Packages

In [1]:
import ydata_profiling
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import tempfile, subprocess
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

# Load parquet file from the Official Repository

In [3]:
# Create temp folder and shallow clone
temp_dir = tempfile.mkdtemp()
subprocess.run(["git", "clone", "--depth", "1", "https://github.com/petrobras/3W.git", temp_dir])

CompletedProcess(args=['git', 'clone', '--depth', '1', 'https://github.com/petrobras/3W.git', 'C:\\Users\\UR8D\\AppData\\Local\\Temp\\tmpqvry16ll'], returncode=0)

# Variables Description


* timestamp = Instant at which observation was generated
* ABER-CKGL = Opening of the GLCK (gas lift choke) [%%]
* ABER-CKP = Opening of the PCK (production choke) [%%]
* ESTADO-DHSV = State of the DHSV (downhole safety valve) [0, 0.5, or 1]
* ESTADO-M1 = State of the PMV (production master valve) [0, 0.5, or 1]
* ESTADO-M2 = State of the AMV (annulus master valve) [0, 0.5, or 1]
* ESTADO-PXO = State of the PXO (pig-crossover) valve [0, 0.5, or 1]
* ESTADO-SDV-GL = State of the gas lift SDV (shutdown valve) [0, 0.5, or 1]
* ESTADO-SDV-P = State of the production SDV (shutdown valve) [0, 0.5, or 1]
* ESTADO-W1 = State of the PWV (production wing valve) [0, 0.5, or 1]
* ESTADO-W2 = State of the AWV (annulus wing valve) [0, 0.5, or 1]
* ESTADO-XO = State of the XO (crossover) valve [0, 0.5, or 1]
* P-ANULAR = Pressure in the well annulus [Pa]
* P-JUS-BS = Downstream pressure of the SP (service pump) [Pa]
* P-JUS-CKGL = Downstream pressure of the GLCK (gas lift choke) [Pa]
* P-JUS-CKP = Downstream pressure of the PCK (production choke) [Pa]
* P-MON-CKGL = Upstream pressure of the GLCK (gas lift choke) [Pa]
* P-MON-CKP = Upstream pressure of the PCK (production choke) [Pa]
* P-MON-SDV-P = Upstream pressure of the production SDV (shutdown valve) [Pa]
* P-PDG = Downhole pressure at the PDG (permanent downhole gauge) [Pa]
* PT-P = Subsea Xmas-tree pressure downstream of the PWV (production wing valve) in the production line [Pa]
* P-TPT = Subsea Xmas-tree pressure at the TPT (temperature and pressure transducer) [Pa]
* QBS = Flow rate at the SP (service pump) [m3/s]
* QGL = Gas lift flow rate [m3/s]
* T-JUS-CKP = Downstream temperature of the PCK (production choke) [oC]
* T-MON-CKP = Upstream temperature of the PCK (production choke) [oC]
* T-PDG = Downhole temperature at the PDG (permanent downhole gauge) [oC]
* T-TPT = Subsea Xmas-tree temperature at the TPT (temperature and pressure transducer) [oC]
* class = Label of the observation
* state = Well operational status

# File per class from the repo

In [4]:
CLASSES = range(0,10)
data_files_by_class = {}
for classe in CLASSES:
  data_files_by_class[classe] = list(Path(temp_dir).rglob(f"*/{classe}/*.parquet"))
  print(f"Found {len(data_files_by_class[classe])} parquet files for class {classe}")

Found 594 parquet files for class 0
Found 128 parquet files for class 1
Found 38 parquet files for class 2
Found 106 parquet files for class 3
Found 343 parquet files for class 4
Found 450 parquet files for class 5
Found 221 parquet files for class 6
Found 46 parquet files for class 7
Found 95 parquet files for class 8
Found 207 parquet files for class 9


# Load and stack data for a specific class


Select class number, load all its parquet files, and concatenate them into a single DataFrame.


In [5]:
CLASSES_NAMES = ['NORMAL', 'ABRUPT_INCREASE_OF_BSW', 'SPURIOUS_CLOSURE_OF_DHSV', 'SEVERE_SLUGGING',
                 'FLOW_INSTABILITY', 'RAPID_PRODUCTIVITY_LOSS', 'QUICK_RESTRICTION_IN_PCK', 'SCALING_IN_PCK',
                 'HYDRATE_IN_PRODUCTION_LINE', 'HYDRATE_IN_SERVICE_LINE']
chosen_class = 2
parquet_files = data_files_by_class[chosen_class]

dfs = []
for file in parquet_files:
  df_temp = pd.read_parquet(file, engine="pyarrow")
  dfs.append(df_temp)

df_combined = pd.concat(dfs, ignore_index=True)
print('Class Selected: ', CLASSES_NAMES[chosen_class])
display(df_combined.head())
display(df_combined.shape)

Class Selected:  SPURIOUS_CLOSURE_OF_DHSV


,ABER-CKGL,ABER-CKP,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,ESTADO-W1,ESTADO-W2,...,PT-P,P-TPT,QBS,QGL,T-JUS-CKP,T-MON-CKP,T-PDG,T-TPT,class,state
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20228170.0,NaN,NaN,90.817520,NaN,NaN,124.5732,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20228170.0,NaN,NaN,90.817520,NaN,NaN,124.5732,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20228170.0,NaN,NaN,90.817529,NaN,NaN,124.5732,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20228170.0,NaN,NaN,90.817530,NaN,NaN,124.5732,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,20228170.0,NaN,NaN,90.817530,NaN,NaN,124.5732,0,0


(737785, 29)

In [6]:
# Numeric summary
print(df_combined.describe(percentiles=[0.01, 0.1, 0.5, 0.9, 0.99]).T)

                  count             mean             std         min  \
ABER-CKGL           0.0              NaN             NaN         NaN   
ABER-CKP        21092.0        31.501981        0.543039    30.88539   
ESTADO-DHSV     10338.0         0.922132        0.267977         0.0   
ESTADO-M1       10338.0              1.0             0.0         1.0   
ESTADO-M2       45248.0         0.694616        0.460575         0.0   
ESTADO-PXO      91147.0         0.015491        0.123498         0.0   
ESTADO-SDV-GL   23747.0         0.502758        0.500003         0.0   
ESTADO-SDV-P    71448.0         0.785102        0.410754         0.0   
ESTADO-W1       31430.0         0.956252        0.204537         0.0   
ESTADO-W2       91142.0           0.0096        0.097511         0.0   
ESTADO-XO       91147.0              0.0             0.0         0.0   
P-ANULAR       261303.0  13007868.558681   4191904.47557   5888008.0   
P-JUS-BS            0.0              NaN             NaN        

# Perform data profiling

Generate a profiling report for the combined DataFrame using `ydata_profiling` or `pandas_profiling`.


In [7]:
try:
    from ydata_profiling import ProfileReport
except ImportError:
    from pandas_profiling import ProfileReport

profile = ProfileReport(df_combined, title=f"3W Data Profiling Report - {CLASSES_NAMES[chosen_class]}", explorative=True)
profile.to_file(f"3w_combined_profile_class_{chosen_class}.html")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

: 

: 

# Visualize Data - One Instance

Create visualizations for the combined DataFrame.

In [ ]:
# Select one instance to visualize
chosen_instance = 25
# Select only numeric columns for plotting that have a reasonable amount of data
# You can adjust the threshold (e.g., 0.5 for at least 50% non-missing values)
threshold = 0.5

In [ ]:
# Plot Chosen Instance
try:
    df = pd.read_parquet(data_files_by_class[chosen_class][chosen_instance], engine="pyarrow")
except Exception as e:
    print(f"Error reading parquet file: {e}")
    print(f"Choose a value between 0 and {len(data_files_by_class[chosen_class]) - 1} for Class {chosen_class}")

# print(f"Loaded {data_files_by_class[chosen_class][chosen_instance]}")

numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
valid_numeric_cols = [col for col in numeric_cols if df[col].notna().sum() / len(df) > threshold]


# Determine the number of subplots needed (one for each valid numeric column)
n_subplots = len(valid_numeric_cols)

if n_subplots == 0:
    print("No valid numeric columns found in the DataFrame to plot based on the non-missing value threshold.")
else:
    # Create subplots
    fig = make_subplots(rows=n_subplots, cols=1, shared_xaxes=True)

    # Add a trace for each valid numeric column to its own subplot
    for i, col in enumerate(valid_numeric_cols):
        fig.add_trace(go.Scattergl(x=df.index, y=df[col], mode='lines', name=col),
                      row=i+1, col=1)

    # Update layout for linked axes and better appearance
    fig.update_layout(
        title=f"Subplots for Numeric variables - Class: {CLASSES_NAMES[chosen_class]} - {str(data_files_by_class[chosen_class][chosen_instance]).split('/')[-1]}",
        hovermode='x unified',
        height=250 * n_subplots # Adjust height based on number of subplots
    )

    # Show the figure
    fig.show()

In [ ]:
pd.options.plotting.backend = "plotly"
df[valid_numeric_cols].plot(title=f"Single Plot Numeric Variables - Class: {CLASSES_NAMES[chosen_class]} - {str(data_files_by_class[chosen_class][chosen_instance]).split('/')[-1]}")

In [ ]:
# Normalize the data in valid_numeric_cols
scaler = MinMaxScaler()
df_normalized = df[valid_numeric_cols].copy()
df_normalized[valid_numeric_cols] = scaler.fit_transform(df_normalized[valid_numeric_cols])

# Create a single plot with normalized values
pd.options.plotting.backend = "plotly"
df_normalized.plot(title=f"Single Plot Normalized Numeric Variables (Class: {CLASSES_NAMES[chosen_class]}, Instance: {chosen_instance}) - - {str(data_files_by_class[chosen_class][chosen_instance]).split('/')[-1]}")